# Alternative names (combinations)

In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

# Autores

In [2]:
%%time

autores_df = pd.read_csv('autores3.csv', header=0)
autores_df = autores_df.fillna('')

CPU times: user 4.79 s, sys: 294 ms, total: 5.08 s
Wall time: 5.08 s


In [3]:
autores_df.shape

(3054360, 5)

In [4]:
autores_df.head()

,TITULO,FECHA_PUB,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
0,Un bicho extraño,2010,Spain,Mon Daporta,DESCONOCIDO
1,Un bicho extraño,2010,Spain,Mon Daporta,DESCONOCIDO
2,Quisiera tener ...,2010,Mexico,Giovanna Zoboli,DESCONOCIDO
3,Johannes Gutenberg,2010,Spain,Lluís Borràs Perelló,DESCONOCIDO
4,La fantástica leyenda de: la princesa y el dragón,2010,Spain,Sonia Alins,DESCONOCIDO


# Autores con país desconocido

In [5]:
autores_pais_desconocido_df = autores_df[
    (autores_df['AUTOR'] != 'DESCONOCIDO') &
    (autores_df['PAIS_AUTOR'] == 'DESCONOCIDO')
]

In [6]:
autores_pais_desconocido_df.shape

(1922405, 5)

In [7]:
autores_pais_desconocido_df.head()

,TITULO,FECHA_PUB,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
0,Un bicho extraño,2010,Spain,Mon Daporta,DESCONOCIDO
1,Un bicho extraño,2010,Spain,Mon Daporta,DESCONOCIDO
2,Quisiera tener ...,2010,Mexico,Giovanna Zoboli,DESCONOCIDO
3,Johannes Gutenberg,2010,Spain,Lluís Borràs Perelló,DESCONOCIDO
4,La fantástica leyenda de: la princesa y el dragón,2010,Spain,Sonia Alins,DESCONOCIDO


In [8]:
autores_pais_desconocido_df['AUTOR'].nunique()

134054

## País autor

In [9]:
def get_pais_autor_data_wkd(name, lang):
    template = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT DISTINCT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author wdt:P31 wd:Q5 .
            {{
                {{
                    ?author wdt:P106 wd:Q36180 .
                }}
                UNION
                {{ 
                    ?author wdt:P106 ?prof .
                    ?prof wdt:P279 wd:Q36180 .
                }}
            }}
            ?author wdt:P19 ?pob .
            ?pob wdt:P17 ?country .
            ?country rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': query, 'format': 'json'}).json()
    return data

In [10]:
sparql_dbp = SPARQLWrapper("http://dbpedia.org/sparql")

def get_pais_autor_data_dbp(name, lang):
    template = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT DISTINCT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author rdf:type dbo:Person .
            {{
                {{
                    ?author rdf:type dbo:Writer .
                }}
                UNION
                {{
                    ?author rdf:type ?prof .
                    ?prof rdfs:subClassOf dbo:Writer .
                }}
            }}
            ?author dbp:birthPlace ?pob .
            ?pob rdf:type dbo:Country .
            ?pob rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    sparql_dbp.setQuery(query)
    sparql_dbp.setReturnFormat(JSON)
    data = sparql_dbp.query().convert()
    return data

In [11]:
def get_pais_autor_data(db, name, lang):
    try:
        if db == 'wdt':
            data = get_pais_autor_data_wkd(name, lang)
        elif db == 'dbp':
            data = get_pais_autor_data_dbp(name, lang)

        results = set(d['countryLabel']['value'] for d in data['results']['bindings'])
        if len(results) == 0:
            result = 'UNKOWN'
        elif len(results) == 1:
            result = results.pop()
        else:
            print('MáS DE UN PAíS PARA', name, ':', results)
            result = results.pop()
    except:
        print('ERROR EN SPARQL PARA', name)
        result = 'UNKOWN'
    return result

In [12]:
def get_pais_autor(name):
    for lang in ['en', 'es']:
        for db in ['wdt', 'dbp']:
            r = get_pais_autor_data(db, name, lang)
            if r != 'UNKOWN':
                return r
    return 'DESCONOCIDO'

In [13]:
%%time

autores_paises_dict = {}

for i, x in autores_df.drop_duplicates(['AUTOR']).iterrows():
    autores_paises_dict[x['AUTOR']] = x['PAIS_AUTOR']

CPU times: user 17.7 s, sys: 5.98 ms, total: 17.7 s
Wall time: 17.7 s


In [14]:
def combinaciones(ls):
    r = []
    for x in ls:
        aux = list(ls)
        aux.remove(x)
        r.append(aux)
    return r

In [15]:
%%time

for autor in autores_pais_desconocido_df['AUTOR'].unique():
    nombres = autor.split()
    if len(nombres) > 2:
        for c in combinaciones(nombres):
            aux = ' '.join(c)
            pais = get_pais_autor(aux)
            if pais != 'DESCONOCIDO':
                autores_paises_dict[autor] = pais
                break

ERROR EN SPARQL PARA Geográfico "Agustín Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Geográfico "Agustín Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Geográfico "Agustín Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Geográfico "Agustín Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Instituto "Agustín Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Instituto "Agustín Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Instituto "Agustín Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Instituto "Agustín Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Instituto Geográfico Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Instituto Geográfico Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Instituto Geográfico Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Instituto Geográfico Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Instituto Geográfico "Agustín Subdirección de Inve
ERROR EN SPARQL PARA Instituto Geográfico "Agustín Subdirección 

In [16]:
%%time

def process_pais(x):
    return autores_paises_dict[x['AUTOR']]

autores_df['PAIS_AUTOR'] = autores_df.apply(process_pais, axis=1)

CPU times: user 1min 16s, sys: 166 ms, total: 1min 16s
Wall time: 1min 16s


# Saving results

In [17]:
autores_df.to_csv('autores4.csv', index=False)